# Aimsun SQLite Output Database Analyzer

This purpose of this Jupyter notebook is to process the database produced by the Aimsun simulations. Run the iPython cell below in order to import necessary imports.

In [63]:
import numpy as np
import scipy
import sqlite3

### SQLite Helper Functions

These functions are wrapper functions for the underlying SQLite commands we use to query data from the SQLite database. The notebook assumes that the Aimsun database file has been placed into the current directory of the notebook.

In [68]:
def create_connection(db_file):
    conn = sqlite3.connect(db_file)
 
    return conn

def select_all_from_table(conn, table, should_print = True):
    cur = conn.cursor()
    
    if should_print:
        # Prevents us from accidentally clogging up the notebook with huge print statements
        cur.execute("SELECT * FROM {} LIMIT 10".format(table))
    else:
        cur.execute("SELECT * FROM {}".format(table))

    rows = cur.fetchall()

    if should_print:
        for row in rows:
            print(row)
    
    return rows

def select_where_from_table(conn, table, clause, should_print = True):
    cur = conn.cursor()
    cur.execute("SELECT * FROM {} WHERE {}".format(table, clauses))
 
    rows = cur.fetchall()
 
    if should_print:
        for row in rows:
            print(row)
    
    return rows

def show_all_tables(conn, should_print = True):
    cur = conn.cursor()
    cur.execute('SELECT name from sqlite_master where type= "table"')
 
    rows = cur.fetchall()
 
    if should_print:
        for row in rows:
            print(row)
    
    return rows

### SimulatorInfo Class (SIM_INFO Table)

This class parses information from the ```SIM_INFO``` table in the Aimsun database. This table stores meta information about the simulation in general, including the owner of the file and the version number currently in use.

In [62]:
class SimulatorInfo:
    
    def __init__(self, values):
        self.data_id = values[0]
        self.data_id_name = values[1]
        self.effective_data_id = values[2]
        
        self.uses_external_id = True if values[3] else False
        
        self.scenario_date = values[4]
        self.start_time = values[5]
        self.duration = values[6]
        
        self.rand_seed = values[7]
        
        self.type = 'Simulated Data' if values[8] == 1 else 'Average'
        self.warm_up_time = values[9]
        
        self.sim_model = values[11]
        self.aimsun_version = values[12]
        self.num_iters = values[13]
        self.exec_date = values[14]
        
        self.experiment_id = values[14]
        self.experiment_name = values[15]
        
        self.scenario_id = values[17]
        self.scenario_name = values[18]
        
        self.author = values[28]

    def __str__(self):
        delimiter = ","
        
        return "Data ID: {}{}".format(self.data_id, delimiter) +\
            "Data ID Name: {}{}".format(self.data_id_name, delimiter) +\
            "Start Time: {}{}".format(self.start_time, delimiter) +\
            "Duration: {}{}".format(self.duration, delimiter) +\
            "Type: {}{}".format(self.type, delimiter) +\
            "Simulation Model: {}{}".format(self.sim_model, delimiter) +\
            "Execution Date: {}{}".format(self.exec_date, delimiter) +\
            "Scenarion Name: {}{}".format(self.scenario_name, delimiter) +\
            "Owner: {}".format(self.author)
    
    def __repr__(self):
        return str(self)

### Lane Class (MILANE Table)

This class parses information from the ```MILANE``` table in the Aimsun database. This table stores microsimulator result statistical information about each lane of the sections for each period.

In [48]:
class Lane:
    
    def __init__(self, values):
        self.replication_id = values[0]
        self.section_id = values[1]
        self.external_id = values[2]
        self.vehicle_type = values[3]
        self.time_interval = values[4]
        self.lane_id = values[5]
        self.vehicle_cnt = values[6]
        self.flow = values[7]
        self.input_vehicle_cnt = values[8]
        self.input_flow = values[9]
        self.density = values[10]
        self.queue_length = values[11]
        self.queue_max_length = values[12]
        self.delay_time = values[13]
        self.queue_waiting_time = values[14]
        self.speed = values[15]
        self.harmonic_speed = values[16]
        self.travel_time = values[17]
        self.stop_time = values[18]
    
    def __str__(self):
        delimiter = ","
        
        return "External ID: {}{}".format(self.external_id, delimeter) +\
            "Time Interval: {}{}".format(self.time_interval, delimeter) +\
            "Vehicle Count: {}{}".format(self.vehicle_cnt, delimeter) +\
            "Avg. Flow: {}{}".format(self.flow, delimeter) +\
            "Avg. Density: {}{}".format(self.density, delimeter) +\
            "Avg. Queue Length: {}{}".format(self.queue_length, delimeter) +\
            "Avg. Delay Time: {}{}".format(self.delay_time, delimeter) +\
            "Avg. Travel Time: {}".format(self.travel_time)
    
    def __repr__(self):
        return str(self)

### Node Class (MINODE Table)

This class parses information from the ```MINODE``` table in the Aimsun database. This table stores microsimulator result statistical information about each node for each period.

In [47]:
class Node:
    
    def __init__(self, values):
        self.replication_id = values[0]
        self.section_id = values[1]
        self.external_id = values[2]
        self.vehicle_type = values[3]
        self.time_interval = values[4]
        self.approach_delay = values[5]
        self.lost_vehicles = values[6]
        self.missed_turnings = values[7]
    
    def __str__(self):
        delimiter = ","
        
        return "External ID: {}{}".format(self.external_id, delimiter) +\
            "Time Interval: {}{}".format(self.time_interval, delimiter) +\
            "Avg. Approach Delay: {}{}".format(self.approach_delay, delimiter) +\
            "Lost Vehicles: {}{}".format(self.lost_vehicles, delimiter) +\
            "Missed Turnings: {}".format(self.missed_turnings)
    
    def __repr__(self):
        return str(self)

### Detector Class (MIDETEC Table)

This class parses information from the ```MIDETEC``` table in the Aimsun database. This table stores microsimulator result statistical information collected by a detector in a road section.

In [46]:
class Detector:
    
    def __init__(self, values):
        self.replication_id = values[0]
        self.section_id = values[1]
        self.external_id = values[2]
        self.vehicle_type = values[3]
        self.time_interval = values[4]
        self.vehicle_count = values[5]
        self.flow = values[6]
        self.speed = values[7]
        self.occupancy = values[8]
        self.density = values[9]
        self.headway = values[10]
    
    def __str__(self):
        delimiter = ","
        
        return "External ID: {}{}".format(self.external_id, delimiter) +\
            "Time Interval: {}{}".format(self.time_interval, delimiter) +\
            "Vehicle Count: {}{}".format(self.vehicle_count, delimiter) +\
            "Flow: {}{}".format(self.flow, delimeter) +\
            "Avg. Speed: {}{}".format(self.speed, delimiter) +\
            "Avg. Occupancy: {}{}".format(self.occupancy, delimiter) +\
            "Avg. Density: {}{}".format(self.density, delimiter) +\
            "Avg. Headway: {}".format(self.headway)
    
    def __repr__(self):
        return str(self)

### Main Helper Functions

To be implemented. The skeleton code has been given below.

In [42]:
def get_link_flow():
    pass

def get_speed():
    pass

def get_travel_time():
    pass

def get_path_flow():
    pass

def compare_flows():
    pass

def compare_speed():
    pass

def compare_travel_time():
    pass

def compare_path_flow():
    pass

## Main Code

This section contains the code that should be run in order to generate results in this notebook. Below is the necessary starter code that connects to the database and outputs the current simulator info:

In [50]:
database = "aimsun-outputs.sqlite"
conn = create_connection(database)
model_info = SimulatorInfo(select_all_from_table(conn, "SIM_INFO", should_print = True)[0])

(27989, 'Replication 27989', 27989, None, 0, '1970-01-02', 50400, 21600, 2343, 1, 0, 'micro', '8.4.0 (Thu May 16 2019 afba3b3059)', 1, '2020-02-23T10:28:05', 27988, 'Micro SRC Experiment 27988', 27981, 'Dynamic Scenario 27981', 2, 36, 2, 36, '{8cd9443d-dd1e-45b4-86eb-de95499503cf}', 27979, 0, 62192, '2020-02-23T10:32:01', 'Theo', None)


The code below generates the lists of nodes, lanes, and detectors.

In [66]:
nodes = []
lanes = []
detectors = []

for row in select_all_from_table(conn, "MINODE", should_print = False):
    nodes.append(Node(row))

for row in select_all_from_table(conn, "MILANE", should_print = False):
    lanes.append(Lane(row))

for row in select_all_from_table(conn, "MIDETEC", should_print = False):
    detectors.append(Detector(row))

In [67]:
len(nodes), len(lanes), len(detectors)

(10416, 35044, 8)